In [2]:
!pip install -q chromadb
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers
!pip install -q langchain
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q bitsandbytes
!pip install -q trl
!pip install -q sentence_transformers
!pip install -q pypdf
!pip install -q unstructured

In [2]:
!transformers-cli env


Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.39.2
- Platform: Windows-10-10.0.22621-SP0
- Python version: 3.11.8
- Huggingface_hub version: 0.22.1
- Safetensors version: 0.4.2
- Accelerate version: 0.28.0
- Accelerate config: 	not found
- PyTorch version (GPU?): 2.2.2+cu121 (True)
- Tensorflow version (GPU?): not installed (NA)
- Flax version (CPU?/GPU?/TPU?): not installed (NA)
- Jax version: not installed
- JaxLib version: not installed
- Using GPU in script?: <fill in>
- Using distributed or parallel set-up in script?: <fill in>



In [2]:
import os
import torch
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import SFTTrainer

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#################################################################
# Tokenizer
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.2'
#model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like mistralai/Mistral-7B-Instruct-v0.2 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [46]:
#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "bfloat16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Device partition
device_map = "auto"

In [47]:
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [48]:
#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    low_cpu_mem_usage = True,
    device_map = device_map
)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.10s/it]


In [30]:
#################################################################
# Base inference
#################################################################

prompt = """En que consiste el Artículo 7 de la ley 11/2022 del BOE de españa"""

# CUDA: Para programar directamente la GPU
model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\mistral\modeling_mistral.py:688: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


En que consiste el Artículo 7 de la ley 11/2022 del BOE de españa?

Artículo 7 de la Ley 11/2022, publicada en el Boletín Oficial del Estado (BOE) de España, establece las condiciones para la aplicación de la exención de IVA en el suministro de energía eléctrica, gas natural y otros gases, así como el suministro de agua potable y de aguas residuales.

Según este artículo, para que una empresa pueda disfrutar de la exención de IVA en el suministro de energía eléctrica, gas natural y otros gases, debe cumplir con las siguientes condiciones:

1. Ser titular de una concesión, autorización o permiso expedido por las competentes autoridades para el suministro de energía eléctrica, gas natural o otros gases.
2. Ser titular de una licencia o permiso expedido por las competentes autoridades para el suministro de agua potable o aguas residuales.
3. No tener por objeto la venta al


## LangChain

In [2]:
import pypdf
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredXMLLoader

CHROMA_PATH = "chroma"
DATA_PATH = "data"

In [3]:
def get_pdf_text(path):
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    
    return pages

def get_xml_text(path):
    loader = UnstructuredXMLLoader(
        path,
        mode = "elements", #Divide el report en cada uno de los campos del xml y crea un Document para cada uno
        strategy="fast",
        metadata_columns=["subtype", "type", "chief_complaint", "admit_diagnosis", "discharge_diagnosis", "year", "downlaod_time", "deid"],
    )
    docs = loader.load()
    return docs

In [34]:
from bs4 import BeautifulSoup
import os

def XMLLoader(folder_path):
    reportData_list = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                content = file.read()

            soup = BeautifulSoup(content, 'xml')

            reportData = {}
            for tag in soup.find_all():
                if tag.name != "report":
                    reportData[tag.name] = tag.text.strip()

            reportData["filename"] = filename
            reportData_list.append(reportData)

    return reportData_list

In [35]:
data =  XMLLoader("data")

In [49]:
from langchain.docstore.document import Document

documents = []
for doc in data:
    
    document = Document(page_content=doc["report_text"], metadata={"source": doc["filename"], 
                                                                   "subtype": doc["subtype"], 
                                                                   "type": doc["type"], 
                                                                   "chief_complaint": doc["chief_complaint"], 
                                                                   "admit_diagnosis": doc["admit_diagnosis"], 
                                                                   "discharge_diagnosis": doc["discharge_diagnosis"], 
                                                                   "year": doc["year"], 
                                                                   "downlaod_time": doc["downlaod_time"], 
                                                                   "deid": doc["deid"]})
    documents.append(document)

In [42]:
#documents[1].metadata["subtype"]="TEST"
#print(documents[7])

### Parent document retriever

In [81]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

parent_docs =  documents

# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

child_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=0,
        length_function=len,
        add_start_index=True
    )


store = InMemoryStore()
vectorstore = Chroma(embedding_function=embeddings, collection_name="fullDoc", persist_directory="medicalReports_DB")

parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    search_kwargs={"k": 3}
)

parent_document_retriever.add_documents(parent_docs, ids=None)

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [82]:
print(f"Parent chunks (Number of reports): {len(list(store.yield_keys()))}")
print(f"Child chunks: {len(parent_document_retriever.vectorstore.get()['ids'])}")

Parent chunks (Number of reports): 33
Child chunks: 2760


In [56]:
# DB keys
parent_document_retriever.vectorstore.get()

{'ids': ['00314b2b-04f1-4ffa-b5be-ba669ea8d506',
  '0080df04-a5a7-4335-b6b5-6b75ac33a3d6',
  '00aae33c-4fea-49d5-a562-55250c445909',
  '00b76dd0-03ee-4e54-ab28-d78ced470dfc',
  '012d3a7b-5032-4c36-bf3c-c96884f0469a',
  '017b3f97-bd19-4e63-b295-ae269074279d',
  '027aa689-bfce-4ae2-9c83-4b3cd9bf6b4c',
  '02b29da3-7597-4c19-a16c-6b072cfec808',
  '03e3274d-2032-473f-ac76-2868e1034395',
  '04067b02-a641-4d13-8d82-81469c6c55c1',
  '0411277e-2c63-48db-9f82-9ae06c10ed77',
  '04340f22-d725-4f97-b5f0-b4c83e93ac60',
  '046de660-3967-4c9a-ae40-b063b0c8d4df',
  '052f40fc-b727-454c-82e6-7af4dccaecd4',
  '053296c6-4947-409a-a69f-61bce906fe40',
  '058f1edb-511d-4376-ac12-61e6f31b3f7d',
  '059e191e-3810-4172-8f04-428807c21dff',
  '0616c9b2-eec6-41da-a1e2-8d4c876c9115',
  '0670d4ee-dffc-49e7-9916-26c6d05dba88',
  '06e0d8ed-185b-47d5-9ce4-e53f7828f580',
  '07134702-576b-4f76-b8c6-8884979c5e33',
  '073ab1a6-de18-46d4-bdfc-8fbf6f7e7b3b',
  '07598418-c4ec-4a91-9c8a-59a3feb022b1',
  '07cb3c20-3c83-4ade-8648-

In [83]:
# RAG

from langchain_core.prompts import ChatPromptTemplate

TEMPLATE = """
[INST]
You are an expert doctor. Use the medical reports provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}

[/INST]
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

def load_model():
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.1,
        repetition_penalty=1.1,
        return_full_text=True,
        max_new_tokens=300,
        do_sample=True
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf

In [51]:
llm = load_model()

In [84]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

setup_and_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": parent_document_retriever })
output_parser = StrOutputParser()

# LCEL language to create the chain (de izquierda a derecha el output de uno se pasa al otro)
parent_retrieval_chain = setup_and_retrieval | rag_prompt | llm | output_parser

In [85]:
out = parent_retrieval_chain.invoke("Wich patient has a history of cholecystectomy")
print(out)

Human: 
[INST]
You are an expert doctor. Use the medical reports provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
Wich patient has a history of cholecystectomy

Context:
[Document(page_content="[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]\n\n\n**INSTITUTION\nConsultation\nName: **NAME[AAA, BBB]\nAcct #: **ID-NUM\nMRN: **ID-NUM\nAdmitted: \nDate: **DATE[Mar 14 2006]\nDict: **NAME[XXX, WWW]\nAttend: **NAME[ZZZ, YYY]\nREASON FOR CONSULT:  Sarcoid, possible pneumonia.\nHISTORY OF PRESENT ILLNESS:  **NAME[BBB AAA] is a **AGE[in 40s]-year-old female who gives \nabout a 16 or 17-year history of sarcoid.  She has had chronic skin rash, and \nbiopsy showed granulomas consistent with sarcoid.  This was in the early \n1999.  These were possibly in 1990.  She later had a breast lump.  Evidently, \nthis showed granulomatous inflammation as well and it was thought to be \nsecondary to sarcoid.  She is currently a